In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow
from copy import copy


# zakres [-0.5, 0.5]
def rectangular_kernel1(x):
    return np.where(np.abs(x) < 0.5, 1, 0)

# zakres [0, 1]
def rectangular_kernel2(x):
    return np.where(np.abs(x - 0.5) < 0.5, 1, 0)

def triangle_kernel(x):
    return np.maximum(0, 1 - np.abs(x))

def sinc_kernel(x):
    return np.where(np.abs(x) < 1e-5, 1, np.sin(np.pi*x) / (np.pi*x))

# sinc cut-off
def sinc_kernel_cut(x, a):
    return np.where(np.abs(x) < 1e-5, 1, np.sin(np.pi * x * a) / (np.pi * x * a))

def keys_kernel(kernel_grid, a):
  kernel = np.zeros_like(kernel_grid)
  for i in range(kernel_grid.shape[0]):
    for j in range(kernel_grid.shape[1]):
      if np.abs(kernel_grid[i,j]) <= 1:
        kernel[i,j] = (a+2) * (np.abs(kernel_grid[i,j])) ** 3 - (a+3) * (np.abs(kernel_grid[i,j])) ** 2 + 1
      elif np.abs(kernel_grid[i,j]) > 1 and np.abs(kernel_grid[i,j]) <= 2:
        kernel[i,j] = a * (np.abs(kernel_grid[i,j])) ** 3 - 5 * a * (np.abs(kernel_grid[i,j])) ** 2 + 8 * a * np.abs(kernel_grid[i,j]) - 4 * a
      else:
        kernel[i,j] = 0

  return kernel


def upscale(img, scale):
  # img = cv2.imread('Pasikonik.jpg')
  # img = cv2.resize(img, (500,500))
  img_x, img_y, channels = img.shape
  img = np.array(img)
  img_new = np.zeros((int(scale*img_x), int(scale*img_y), channels))

  x = np.arange(img_y)
  x_interp = np.arange(scale*img_y)
  x_interp = np.linspace(0,img_y-1,int(img_x*scale))
  period = x[1] - x[0] # store  period  between  samples
  kernel_grid = np.tile(x_interp, (len(x), 1))
  offsets = np.tile(np.expand_dims(x, axis=-1), (1, len(x_interp)))
  kernel_grid = (kernel_grid - offsets) / period
  kernels = keys_kernel(kernel_grid,-1)


  for c in range(channels):
    for i in range(img_y):
      img_new[i,:,c] = np.dot(img[i,:,c],kernels)

    for j in range(int(img_x*scale)):
      img_new[:,j,c] = np.dot(img_new[0:img_x,j,c],kernels)


  cv2_imshow(img_new)
  print(img_new.shape)
  return img_new



def downscale(scale):
  img = cv2.imread('Pasikonik.jpg')
  img = cv2.resize(img, (500,500))
  img_x, img_y, channels = img.shape
  img = np.array(img)
  img_new = np.zeros((int(scale*img_x), int(scale*img_y), channels))
  img_copy = copy(img)

  x = np.arange(img_y)
  x_interp = np.linspace(0,img_y-1,int(img_x*scale))

  period = x[1] - x[0] # store  period  between  samples
  kernel_grid = np.tile(x_interp, (len(x), 1))
  offsets = np.tile(np.expand_dims(x, axis=-1), (1, len(x_interp)))
  kernel_grid = (kernel_grid - offsets) / period
  kernels = keys_kernel(kernel_grid,-1)

  for c in range(channels):
    for i in range(img_y):#(min(img_y, int(img_y*scale))):
      img_copy[i,0:int(img_y*scale),c] = np.dot(img[i,:,c],kernels)

    for j in range(int(img_x*scale)):#(max(img_x, int(img_x*scale))):
      img_new[:,j,c] = np.dot(img_copy[0:img_x,j,c],kernels)


  return img_new
  # cv2_imshow(img)
  cv2_imshow(img_new)
  print(img_new.shape)

img = downscale(0.5)
img = upscale(img,2)



